but : prédire les zone de fragilité de ARNm

à partir de : séquence d'arn, les appariements prédis de structure secondaire  et quel méthode utilisée
en input on a  le jeux de données anoté mais pas complet. 

On part des réseaux récurrents: on a un input et un output, l'idée est qu'on va réinjecter de l'info de l'input précedent dans l'input présent. (réseau neuromemetique saint)



# Importation des modules

In [384]:
%%capture

import shutil
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import tensorflow as tf
from multiprocessing import Pool

import os
import gc
import json
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import layers as L
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical



# Récupération et exploitation des données

In [385]:
#ouvrir les fichiers json test et train 
train = pd.read_json('Data/train.json', lines = True)  # 2400rows * 19columns
print("longueur fichier train non filtré {}".format(len(train)))

# filter le signal bruit, ne prendre que les valeurs > 1.
train= train[train.SN_filter == 1] #filter le signal bruit ::: 1589 rows 
print("longueur fichier train  filtré {}".format(len(train)))

test = pd.read_json('Data/test.json', lines = True)
print("longueur fichier test {}".format(len(test)))

sub = pd.read_csv("Data//sample_submission.csv") #fichier sortie cible


longueur fichier train non filtré 2400
longueur fichier train  filtré 1589
longueur fichier test 3634


In [386]:
df_check_nan=train.isnull().values.any()
print("null result: ", df_check_nan)

df_check_na=  train.isna().values.any()
print("na result: ", df_check_na)



null result:  False
na result:  False


In [387]:
train.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
5,5,id_00ab2d761,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...,4.136,1,107,68,"[0.1942, 0.2041, 0.1626, 0.1213, 0.10590000000...","[0.2726, 0.2984, 0.21660000000000001, 0.1637, ...","[0.3393, 0.2728, 0.2005, 0.1703, 0.1495, 0.134...","[0.165, 0.20520000000000002, 0.179, 0.1333, 0....","[0.2864, 0.24710000000000001, 0.2222, 0.1903, ...","[0.7642, 1.6641, 1.0622, 0.5008, 0.4107, 0.133...","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[1.9554, 2.1298, 1.0403, 0.609, 0.5486, 0.386,...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0..."
6,6,id_00abef1d7,GGAAAACAAUUGCAUCGUUAGUACGACUCCACAGCGUAAGCUGUGG...,.........((((((((......((((((((((((....)))))))...,EEEEEEEEESSSSSSSSIIIIIISSSSSSSSSSSSHHHHSSSSSSS...,2.485,1,107,68,"[0.422, 0.5478000000000001, 0.4749000000000000...","[0.4801, 0.7943, 0.42160000000000003, 0.397300...","[0.9822000000000001, 1.272, 0.6940000000000001...","[0.5827, 0.7555000000000001, 0.5949, 0.4511, 0...","[0.9306000000000001, 1.0496, 0.5844, 0.7796000...","[0.895, 2.3377, 2.2305, 2.003, 1.9006, 1.0373,...","[0.46040000000000003, 3.6695, 0.78550000000000...","[2.7711, 7.365, 1.6924000000000001, 1.43840000...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1..."
7,7,id_00b436dec,GGAAAUCAUCGAGGACGGGUCCGUUCAGCACGCGAAAGCGUCGUGA...,.....(((((((((((..(((((((((..((((....))))..)))...,EEEEESSSSSSSSSSSIISSSSSSSSSIISSSSHHHHSSSSIISSS...,1.727,1,107,68,"[0.4843, 0.5233, 0.4554, 0.43520000000000003, ...","[0.8719, 1.0307, 0.6649, 0.34500000000000003, ...","[0.7045, 0.7775000000000001, 0.5662, 0.4561, 0...","[0.384, 0.723, 0.4766, 0.30260000000000004, 0....","[0.7429, 0.9137000000000001, 0.480400000000000...","[1.1576, 1.5137, 1.3382, 1.5622, 1.2121, 0.295...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.8641, 2.3767, 1.149, 1.0132, 0.9876, 0.0, 0...","[0.49060000000000004, 4.6339, 1.95860000000000...","[1.2852000000000001, 2.5460000000000003, 0.234..."


In [388]:
test.head()

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [389]:
train.shape, test.shape

((1589, 19), (3634, 7))

In [390]:
print(train.describe(), "\n")

print(train.info())

             index  signal_to_noise  SN_filter  seq_length  seq_scored
count  1589.000000      1589.000000     1589.0      1589.0      1589.0
mean   1198.047199         5.402215        1.0       107.0        68.0
std     698.720089         2.524798        0.0         0.0         0.0
min       0.000000         0.993000        1.0       107.0        68.0
25%     585.000000         3.484000        1.0       107.0        68.0
50%    1198.000000         5.222000        1.0       107.0        68.0
75%    1811.000000         6.853000        1.0       107.0        68.0
max    2399.000000        17.194000        1.0       107.0        68.0 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1589 entries, 0 to 2399
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                1589 non-null   int64  
 1   id                   1589 non-null   object 
 2   sequence             1589 non-null   object 
 3

In [391]:
print(test.describe(), "\n")

print(test.info())

             index   seq_length   seq_scored
count  3634.000000  3634.000000  3634.000000
mean   1816.500000   126.018987    87.018987
std    1049.189767     8.702624     8.702624
min       0.000000   107.000000    68.000000
25%     908.250000   130.000000    91.000000
50%    1816.500000   130.000000    91.000000
75%    2724.750000   130.000000    91.000000
max    3633.000000   130.000000    91.000000 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3634 entries, 0 to 3633
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                3634 non-null   int64 
 1   id                   3634 non-null   object
 2   sequence             3634 non-null   object
 3   structure            3634 non-null   object
 4   predicted_loop_type  3634 non-null   object
 5   seq_length           3634 non-null   int64 
 6   seq_scored           3634 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 198.

# Préparation des données pour le réseau: 

## <span style="color: #39B300"> 1. Input x (train + test) : 

In [392]:
col_names = ['sequence', 'structure', 'predicted_loop_type']
X_train = train[col_names]
X_train.shape
X_train.head()


,sequence,structure,predicted_loop_type
0,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...
2,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...
5,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...
6,GGAAAACAAUUGCAUCGUUAGUACGACUCCACAGCGUAAGCUGUGG...,.........((((((((......((((((((((((....)))))))...,EEEEEEEEESSSSSSSSIIIIIISSSSSSSSSSSSHHHHSSSSSSS...
7,GGAAAUCAUCGAGGACGGGUCCGUUCAGCACGCGAAAGCGUCGUGA...,.....(((((((((((..(((((((((..((((....))))..)))...,EEEEESSSSSSSSSSSIISSSSSSSSSIISSSSHHHHSSSSIISSS...


In [393]:
# On ne prend que les séquence de taille = 107pb ----> public test sur Kaggle
X_test = test.query('seq_length == 107')[col_names]    #output 68
X_test.shape

(629, 3)

In [394]:
"""
# On ne l'utilise pas pour l'instant mais juste au cas ou. -----> Private test sur Kaggle
X_test_priv = test.query('seq_length == 130')[col_names]   #91 de output
X_test_priv.shape
"""

"\n# On ne l'utilise pas pour l'instant mais juste au cas ou. -----> Private test sur Kaggle\nX_test_priv = test.query('seq_length == 130')[col_names]   #91 de output\nX_test_priv.shape\n"

### <span style="color: #26B260"> Hot-coding et reglage de longueure de séquence: 

In [395]:
def encodage(X_strc, X_loop, X_seq, length: int):
    """
    Cette fonction permet le Hot-coding de structure, sequence et loop type.
       Ensuite elle permet de concaténer les résultats.
    Parametre:
    -----
    X_strc: structures secondaire
    X_loop: predected_loop_type
    X_seq : sequences
    length: int , longeure de séquence souhaitée.
    Return:
    -----
    result: np array encoder et concaténer. 
    """
    result = []

    #STRUCTURES:
        # '(': 0
        # ')': 1 
        # '.': 2
    structures = {'(': 0, ')': 1, '.': 2}
    x_code_struc = []

    # predicted_loop_type
        # 'S': 0 
        # 'E': 1 
        # 'B': 2 
        # 'H': 3 
        # 'M': 4 
        # 'I': 5 
        # 'X': 6
    dico_loops = {'S': 0, 'E': 1, 'B': 2, 'H': 3, 'M': 4, 'I': 5, 'X': 6}
    x_code_loops = []

    # Sequence       
        # 'A': 0
        # 'U': 1
        # 'G': 2
        # 'C': 3
    dico_seq = {'A': 0, 'U': 1, 'G': 2, 'C': 3}
    x_code_seq = []
    
    for stru in X_strc:
        liste_stru = []
        for caract in stru:
            liste_stru.append(structures[caract])
        x_code_struc.append(to_categorical(liste_stru, num_classes=3, dtype=int))

    for loops in X_loop:
        liste_loops = []
        for loop_type in loops:
            liste_loops .append(dico_loops[loop_type])
        x_code_loops.append(to_categorical(liste_loops , num_classes=7, dtype=int))

    for seq in X_seq:
        liste_seq = []
        for base in seq:
            liste_seq.append(dico_seq[base])
        x_code_seq.append(to_categorical(liste_seq, num_classes=4, dtype=int))
        
    for i in range(len(x_code_seq)):
        result.append(np.concatenate((x_code_seq[i][0:length], x_code_struc[i][0:length], x_code_loops[i][0:length]), axis=1))
    return np.array(result)

In [396]:

X_train = encodage(X_train['structure'], X_train['predicted_loop_type'], X_train['sequence'], 68)
print("X_train {} :".format(X_train.shape))

X_test = encodage(X_test['structure'], X_test['predicted_loop_type'], X_test['sequence'],68)
print("X_test {}".format(X_test.shape))


X_train (1589, 68, 14) :
X_test (629, 68, 14)


Les données sont sous forme matricielle:
    

pour cette partie, partant des séquences de longueure 170pb on arrive à 68pb (données public):
    
    
    (nombre de seq ARN, taille des seq, nb classes(4+3+7))
    
    
TRAIN:  
    
        - 1589 sequences dARN, longueure: 68 pb , 14 classes)
        
TEST: 

        - 1589 sequences dARN, longueure: 68 pb , 14 classes)

## <span style="color: #39B300"> 2.Ouput y (train): 

In [458]:
col_names = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
Y_train = train[col_names]
Y_train.shape

(1589, 5)

In [459]:
Y_train.head()

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,"[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
2,"[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
5,"[0.7642, 1.6641, 1.0622, 0.5008, 0.4107, 0.133...","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[1.9554, 2.1298, 1.0403, 0.609, 0.5486, 0.386,...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0..."
6,"[0.895, 2.3377, 2.2305, 2.003, 1.9006, 1.0373,...","[0.46040000000000003, 3.6695, 0.78550000000000...","[2.7711, 7.365, 1.6924000000000001, 1.43840000...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1..."
7,"[1.1576, 1.5137, 1.3382, 1.5622, 1.2121, 0.295...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.8641, 2.3767, 1.149, 1.0132, 0.9876, 0.0, 0...","[0.49060000000000004, 4.6339, 1.95860000000000...","[1.2852000000000001, 2.5460000000000003, 0.234..."


In [460]:
def y_to_array(y_train, col_names): 
    Y = np.asarray(y_train)
    size = len(Y[0])
    converted = np.zeros((len(Y),len(Y[0][0]),len(col_names)))
    for i in range(len(col_names)):
        for j in range(size):
            Y[i,j] = np.asarray(Y[i,j])
        for i in range(len(Y)):
            converted[i] = np.vstack((Y[i,0], Y[i,1], Y[i,2], Y[i,3], Y[i,4]))[:,:].T
    return converted

In [461]:
Y_train = y_to_array(Y_train, col_names)

In [462]:
Y_train.shape

(1589, 68, 5)

In [465]:
# 1589 sequences, 68 nucléotides donc pr chaucune on a 5 variables 

## Définir les variables cibles de la prédiction 

In [418]:
targets = list(sub.columns[1:])
print(targets)

# On a 5 columns mais on ne s'interesse que aux columns : 'reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'
# les 2 columns restantes seront remplies par des 0 car elles ne nous interessent pas dans ce projet.

['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
